In [1]:
###################################################################################################
#
# Copyright (C) 2023 Maxim Integrated Products, Inc. All Rights Reserved.
#
# Maxim Integrated Products, Inc. Default Copyright Notice:
# https://www.maximintegrated.com/en/aboutus/legal/copyrights.html
#
###################################################################################################

import os
import sys

import importlib
import numpy as np
import torch

import matplotlib.patches as patches
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'models'))

import ai8x

from datasets import nilm

In [2]:
from pathlib import Path

In [3]:
data_path = Path('../data/NILM')
args = None
# data_path.exists()

train_set, test_set = nilm.ukdale_get_datasets((data_path, args), load_train=True, load_test=True)

In [4]:
batch_size = 32
num_workers = 5

In [5]:
dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)

In [6]:
loaded_data = [batch for batch in dataloader]

In [7]:
batch_0 = loaded_data[0]

In [8]:
mod = importlib.import_module("ai87net-unetnilm")

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working with device:", device)

ai8x.set_device(device=85, simulate=True, round_avg=False)
qat_policy = {'start_epoch': 10, 'weight_bits': 8, 'bias_bits': 8}

Working with device: cuda
Configuring device: MAX78000, simulate=True.


In [18]:
from torch import nn

In [19]:
in_size = 1
output_size = 5
n_layers = 5
features_start = 16

In [23]:
class Conv1D(nn.Module):
    
    def __init__(self,
                 num_channels,
                 num_kernels,
                 kernel_size=3,
                 stride=2,
                 padding=1,
                 last=False,
                 activation="ReLU",
                 batchnorm="NoAffine",
                 **kwargs):
        super(Conv1D, self).__init__()
        
        if not last:
            if activation == "ReLU":
                self.net = ai8x.FusedConv1dBNReLU(in_channels=num_channels,
                                                  out_channels=num_kernels,
                                                  kernel_size=kernel_size,
                                                  stride=stride,
                                                  padding=padding,
                                                  bias=True,
                                                  batchnorm="NoAffine",
                                                  **kwargs)
            elif activation == "Abs":
                self.net = ai8x.FusedConv1dBNAbs(in_channels=num_channels,
                                                  out_channels=num_kernels,
                                                  kernel_size=kernel_size,
                                                  stride=stride,
                                                  padding=padding,
                                                  bias=True,
                                                  batchnorm="NoAffine",
                                                  **kwargs)
            else:
                self.net = ai8x.Conv1d(
                    in_channels=num_channels,
                    out_channels=num_kernels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    batchnorm=batchnorm,
                    **kwargs
                )
        else:
            self.net = ai8x.Conv1d(
                in_channels=num_channels,
                out_channels=num_kernels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                batchnorm=batchnorm,
                **kwargs
            )
        nn.utils.weight_norm(self.net.op)    
        nn.init.xavier_uniform_(self.net.op.weight)
                
    def forward(self, x):
        return self.net(x)

class UNetCNN1D(nn.Module):
    
    def __init__(
            self, 
            num_classes: int = 5,
            num_channels: int = 1,
            **kwargs
    ):
        super().__init__()

        self.prep0 = Conv1D(num_channels=num_channels, num_kernels=64, kernel_size=1,
                            stride=1, padding=0, activation="ReLU", batchnorm="NoAffine", last=False, **kwargs)
        
    def forward(self, x):
        x = self.prep0(x)
        return x


In [24]:
class AI85UNetNILM(nn.Module):
    """
    Large size UNet model. This model also enables the use of folded data.
    """
    def __init__(
            self,
            num_classes=4,         # in_size
            num_channels=48,        #
            # dimensions=(88, 88),  # pylint: disable=unused-argument
            **kwargs
    ):
        super().__init__()

        self.unet = UNetCNN1D(num_classes=num_classes,
                              num_channels=num_channels)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):  # pylint: disable=arguments-differ
        """Forward prop"""
        # Run CNN
        B = x.size(0)
        x = x.permute(0,2,1)

        x = self.dropout(self.unet(x))

        return x


In [25]:
model = AI85UNetNILM(num_channels=in_size,
                     num_classes=output_size)
# model = mod.AI85UNetNILM(num_classes=output_size, num_channels=in_size)

model(batch_0[0])

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 